In [229]:
import sqlite3
import pandas as pd

# Bağlantı
connection = sqlite3.connect("okul_sistemi.db")
cursor = connection.cursor()

# İlişkisel bütünlük için gerekli (Foreign Key hatası almamak için)
cursor.execute("PRAGMA foreign_keys = ON;")

# ---------------------------------------------------------
# 1. TABLOLARI OLUŞTURMA (CREATE)
# ---------------------------------------------------------

# Önce varsa eskileri silelim (Sıralama önemli: En bağımlı olandan başlanır)
cursor.execute("DROP TABLE IF EXISTS donemders")
cursor.execute("DROP TABLE IF EXISTS ogrenciler")
cursor.execute("DROP TABLE IF EXISTS dersler")
cursor.execute("DROP TABLE IF EXISTS bolumler")

# 1. Bölümler Tablosu
cursor.execute("""
CREATE TABLE bolumler (
    id INTEGER PRIMARY KEY,
    bolumadi TEXT NOT NULL
);
""")

# 2. Dersler Tablosu
cursor.execute("""
CREATE TABLE dersler (
    id INTEGER PRIMARY KEY,
    dersadi TEXT NOT NULL,
    donem INTEGER
);
""")

# 3. Öğrenciler Tablosu (Bölümlere bağlı)
cursor.execute("""
CREATE TABLE ogrenciler (
    id INTEGER PRIMARY KEY,
    adisoyadi TEXT NOT NULL,
    bolumid INTEGER,
    tip TEXT, -- Örn: 'Normal', 'İkinci Öğretim'
    FOREIGN KEY (bolumid) REFERENCES bolumler(id)
);
""")

# 4. Dönem Dersleri / Notlar Tablosu (Öğrenci ve Derslere bağlı)
cursor.execute("""
CREATE TABLE donemders (
    id INTEGER PRIMARY KEY,
    dersid INTEGER,
    ogrenciid INTEGER,
    notu INTEGER,
    FOREIGN KEY (dersid) REFERENCES dersler(id),
    FOREIGN KEY (ogrenciid) REFERENCES ogrenciler(id)
);
""")

print("Tablolar başarıyla oluşturuldu.")

# ---------------------------------------------------------
# 2. VERİ EKLEME (INSERT)
# ---------------------------------------------------------

# Bölümler
bolum_data = [
    (1, 'Bilgisayar Mühendisliği'),
    (2, 'Endüstri Mühendisliği'),
    (3, 'Mimarlık')
]
cursor.executemany("INSERT INTO bolumler VALUES (?,?)", bolum_data)

# Dersler
ders_data = [
    (101, 'Algoritma', 1),
    (102, 'Fizik I', 1),
    (103, 'Veritabanı', 3),
    (104, 'İstatistik', 3)
]
cursor.executemany("INSERT INTO dersler VALUES (?,?,?)", ders_data)

# Öğrenciler (bolumid yukarıdaki 1,2,3'ten biri olmalı)
ogrenci_data = [
    (1, 'Ahmet Yılmaz', 1, 'Normal'),     # Bilgisayar
    (2, 'Ayşe Demir', 1, 'İkinci'),       # Bilgisayar
    (3, 'Mehmet Öz', 2, 'Normal'),        # Endüstri
    (4, 'Fatma Kara', 3, 'Normal'),       # Mimarlık
    (5, 'Can Vural', 1, 'Normal')         # Bilgisayar
]
cursor.executemany("INSERT INTO ogrenciler VALUES (?,?,?,?)", ogrenci_data)

# Dönem Dersleri / Notlar (Kim hangi dersi aldı ve kaç aldı?)
not_data = [
    (1, 101, 1, 85),  # Ahmet, Algoritma, 85
    (2, 103, 1, 90),  # Ahmet, Veritabanı, 90
    (3, 101, 2, 45),  # Ayşe, Algoritma, 45
    (4, 103, 2, 70),  # Ayşe, Veritabanı, 70
    (5, 104, 3, 100), # Mehmet, İstatistik, 100
    (6, 102, 5, 60)   # Can, Fizik, 60
]
cursor.executemany("INSERT INTO donemders VALUES (?,?,?,?)", not_data)

connection.commit()
print("Veriler başarıyla eklendi.")

#SORGU FONKSIYON TANIMLA
def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None


Tablolar başarıyla oluşturuldu.
Veriler başarıyla eklendi.


In [230]:
run_query('select*from bolumler')

   id                 bolumadi
0   1  Bilgisayar Mühendisliği
1   2    Endüstri Mühendisliği
2   3                 Mimarlık


In [231]:
run_query('select*from dersler')

    id     dersadi  donem
0  101   Algoritma      1
1  102     Fizik I      1
2  103  Veritabanı      3
3  104  İstatistik      3


In [232]:
run_query('select*from ogrenciler')

   id     adisoyadi  bolumid     tip
0   1  Ahmet Yılmaz        1  Normal
1   2    Ayşe Demir        1  İkinci
2   3     Mehmet Öz        2  Normal
3   4    Fatma Kara        3  Normal
4   5     Can Vural        1  Normal


In [233]:
run_query('select*from donemders')

   id  dersid  ogrenciid  notu
0   1     101          1    85
1   2     103          1    90
2   3     101          2    45
3   4     103          2    70
4   5     104          3   100
5   6     102          5    60


In [234]:
#Kim, hangi bölümde okuyor, hangi dersi almış ve kaç puan almış
run_query("""
    SELECT o.adisoyadi, b.bolumadi, d.dersadi, n.notu
    FROM donemders n
    JOIN ogrenciler o ON n.ogrenciid = o.id
    JOIN dersler d ON n.dersid = d.id
    JOIN bolumler b ON o.bolumid = b.id;
""")

      adisoyadi                 bolumadi     dersadi  notu
0  Ahmet Yılmaz  Bilgisayar Mühendisliği   Algoritma    85
1  Ahmet Yılmaz  Bilgisayar Mühendisliği  Veritabanı    90
2    Ayşe Demir  Bilgisayar Mühendisliği   Algoritma    45
3    Ayşe Demir  Bilgisayar Mühendisliği  Veritabanı    70
4     Mehmet Öz    Endüstri Mühendisliği  İstatistik   100
5     Can Vural  Bilgisayar Mühendisliği     Fizik I    60


In [235]:
#her dersin en yüksek not alan öğrencisini yazınız
run_query("""
    SELECT
        d.dersAdi,
        o.adisoyadi,
        max(dd.notu) as en_iyi_not
    from donemders as dd
    join ogrenciler as o on dd.ogrenciid = o.id
    join dersler as d on dd.dersid = d.id
    group by d.dersAdi;
""")

      dersadi     adisoyadi  en_iyi_not
0   Algoritma  Ahmet Yılmaz          85
1     Fizik I     Can Vural          60
2  Veritabanı  Ahmet Yılmaz          90
3  İstatistik     Mehmet Öz         100


In [236]:
#her dersin en yüksek not alan öğrencisini yazınız
run_query("""
    SELECT d.dersAdi, o.adisoyadi, dd.notu
    FROM donemders dd
    JOIN dersler d ON dd.dersid = d.id
    JOIN ogrenciler o ON dd.ogrenciid = o.id
    WHERE dd.notu = (
        SELECT MAX(notu) 
        FROM donemders 
        WHERE dersid = dd.dersid
        );
""")

      dersadi     adisoyadi  notu
0   Algoritma  Ahmet Yılmaz    85
1  Veritabanı  Ahmet Yılmaz    90
2  İstatistik     Mehmet Öz   100
3     Fizik I     Can Vural    60


In [237]:
#tüm bolümlerin öğrenci sayılarını bolumadı, ogrecni_sayısı şeklinde sıralayınız 
run_query("""
    SELECT 
        A.bolumadi, 
        count(B.id) as ogrenci_sayısı
    FROM bolumler as A        
    LEFT JOIN ogrenciler as B on A.id = B.bolumid
    GROUP BY A.bolumadi;
""")#A⟕B

                  bolumadi  ogrenci_sayısı
0  Bilgisayar Mühendisliği               3
1    Endüstri Mühendisliği               1
2                 Mimarlık               1


In [238]:
#her öğrencinin aldığı ders adedini adsoyad dersadedi şeklinde listeleyiniz
run_query("""
    select
        o.adisoyadi as ad_soyad,
        count(d.ogrenciid) as aldığı_ders_adedi
    from ogrenciler as o
    left join donemders as d on o.id = d.ogrenciid
    group by o.adisoyadi;
""")

       ad_soyad  aldığı_ders_adedi
0  Ahmet Yılmaz                  2
1    Ayşe Demir                  2
2     Can Vural                  1
3    Fatma Kara                  0
4     Mehmet Öz                  1


In [ ]:
#geçme notu 60 ve üzeri olduğuna göre 1 derstene geçen öğrenci adedini bulunuz
run_query("""
    select
        count(dersid)
    from donemders as d
    where notu>60
    group by id;
""")

   count(dersid)
0              4


In [240]:
#her öğrencinin başaraılı ve başarısız olduğu ders saylarını öğrenciadı,basarılı,basarısız şeklinde listeleyiniz

In [241]:
connection.commit()
connection.close()